# Assumptions

- Equidistanced spatial grid

In [1]:
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt

In [2]:
t0 = 0.0
Nt = 11
dt = 0.05

In [3]:
hbar, m = 1.0, 1.0

In [4]:
Nx = 101
x_arr = np.linspace(0, 5.1, Nx)
dx = x_arr[1] - x_arr[0]

Vx = np.zeros_like(x_arr, dtype=float)

wf_x_t0 = np.empty_like(x_arr, dtype=complex)
wf_x_t0[:] = np.sin(pi/5.1*x_arr)
wf_x_t0 /= np.sqrt(np.trapz(np.square(np.abs(wf_x_t0)), x_arr))

In [5]:
from tdse.prop.propagator import Time_Indep_Hamil_Propagator

prop = Time_Indep_Hamil_Propagator(Nx-2, dx, Vx[1:-1], m=1.0)

In [6]:
xp_t0_arr = np.array([0.3, 0.6, 0.78])

#### Initialize

In [7]:
# %matplotlib notebook

# from matplotlib.animation import FuncAnimation

In [8]:
# fig, ax = plt.subplots()

# l_real, = ax.plot([],[])
# l_abs, = ax.plot([],[])
# ax.set_xlim(*x_arr[[0,-1]]), ax.set_ylim(-1,1)
# text = ax.text(0,0,'')

# def init(): 
#     wf_x[:] = wf_x_t0
#     l_real.set_data(x_arr, wf_x.real)
#     l_abs.set_data(x_arr, np.abs(wf_x))
#     return l_real, l_abs
    
# def update(i):
#     prop.propagate(wf_x[1:-1], dt, Nt=10)
#     l_real.set_data(x_arr, wf_x.real)
#     l_abs.set_data(x_arr, np.abs(wf_x))
#     text.set_text(i)

# ani = FuncAnimation(fig, update, frames=15, init_func=init)

In [9]:
from warnings import filterwarnings
filterwarnings('error')

In [10]:
# Initialize
xp_arr = np.empty_like(xp_t0_arr)
xp_arr[:] = xp_t0_arr
xp_arr_t = np.empty((Nt, xp_arr.size))
xp_arr_t[0,:] = xp_arr

wf_x = np.empty_like(wf_x_t0)
wf_x[:] = wf_x_t0
wf_x_t = np.empty((Nt,wf_x.size), dtype=wf_x.dtype)
wf_x_t[0,:] = wf_x


# Arrays for Finite Difference method
A = np.empty((2,2), dtype=float)
A[:,0] = 1.0
b = np.empty((2,), dtype=complex)


# iteration over time
from scipy.optimize import root


for it in range(Nt):
#     print("time-index: {}".format(it))
    prop.propagate(wf_x[1:-1], dt, Nt=1)
    
    for ip, xp in enumerate(xp_arr):
#         print("particle-index: {} / particle-position: {}".format(ip, xp))
        if not (x_arr[0] < xp and xp < x_arr[-1]): continue

            
        def _g_func(_dxp, _xp, _dx, _x0, _wf_arr, _dt, _A, _b, _hbar, _m):
            
            _x = _xp + _dxp
            
            _is0 = int((_x-_x0)//_dx)
#             print("{:.32e}".format(_x),_dxp,_is0, (_x-_x0)%_dx)
            _A[0,1] = - ((_x-_x0)%_dx)
            _A[1,1] = _A[0,1] + _dx
            _b[0] = _wf_arr[_is0]
            _b[1] = _wf_arr[_is0+1]
            
            try: _wf, _dx_wf = np.linalg.solve(_A, _b)
            except np.linalg.LinAlgError as e:
                raise RuntimeError("Failed to get wf at given x: {}".format(_x))
            except: raise Exception("Unexpected error")
                
            if _wf == 0: raise NotImplementedError
                
            _v = _hbar / _m * np.imag(_dx_wf / _wf)
            
            _g = -_dxp + _dt * _v
            return _g
        
#         _g_args = (xp, dx, x_arr[0], wf_x, dt, A, b, hbar, m)
#         print(_g_func(0.0, *_g_args), A, b)
#         print(_g_func(-2.88448088e-33, *_g_args), A, b)
        
#         def callback(dxp, g):
#             _v_at_xp_dxp_t_dt = (g + dxp) / dt
#             print("dxp: {} & _g: {} & xp: {} & _v_at_xp_dxp_t_dt: {}".format(dxp,g,xp,_v_at_xp_dxp_t_dt))

        try: _sol = root(_g_func, 0.0, _g_args, method='hybr', options={})
        except: 
            _msg = "Failed for time-index={}, particle-position={}, opt_result: "
            raise Exception(_msg.format(it, xp))
        if not _sol.success: raise Exception
        _dxp = _sol.x
        xp_arr[ip] += _dxp
        
        
        # Store
        xp_arr_t[it,:] = xp_arr
        wf_x_t[it,:] = wf_x

Exception: Failed for time-index=0, particle-position=0.3, opt_result: 

In [ ]:
t_arr = t0 + dt*np.arange(Nt)

In [ ]:
fig, ax = plt.subplots()
for ip in range(xp_t0_arr.size):
    ax.plot(xp_arr_t[:,ip], t_arr, '.-')
ax.set_xlim(*x_arr[[0,-1]])
ax.set_xlabel("position x / a.u."), ax.set_ylabel("time / a.u.")
fig.tight_layout()
# fig.savefig("particles-test-0.png")

# [TODO] Let's just use jacobian instead of approximation, since it isn't too hard to evaluate